In [49]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [50]:
G = pickle.load(open('graphs/graph.pkl','rb'))

In [51]:
G_processed = nx.convert_node_labels_to_integers(G)


In [52]:
G_processed.remove_edges_from([(u,v) for u,v,d in G_processed.edges(data=True) if d['Plant'] == 0])

In [53]:
edge_attr = np.zeros((len(G_processed.edges()),1))

for i,e in enumerate(G_processed.edges(data=True)):
    # edge_attr[i] = np.array(list(e[2].values()),dtype=float)
    edge_attr[i] = list(e[2].values())[0]

#edge_attr = edge_attr.transpose()


In [54]:
d2d = pd.read_csv('Raw Dataset/Homogenoeus/Temporal Data/Weight/Delivery To distributor.csv')

In [55]:
duration = len(d2d)
node_feature_sequence = []
node_target_sequence = []
for i in range(duration):
    node_features = []
    node_targets = []
    for n,d in G_processed.nodes(data=True):
        node_features.append(np.array([d['delivered'][i],d['sales_order'][i],d['factory issue'][i]]))
        node_targets.append(d['production'][i])
    node_feature_sequence.append(np.array(node_features))
    node_target_sequence.append(np.array(node_targets))
    
    

In [56]:
np.shape(node_feature_sequence)

(221, 28, 3)

In [57]:
np.shape(node_target_sequence)

(221, 28)

In [58]:
np.array(list(G_processed.edges())).transpose().max()

27

In [59]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

data_iter = StaticGraphTemporalSignal(
    edge_index= np.array(list(G_processed.edges())).transpose(),
    edge_weight = edge_attr,
    features = node_feature_sequence,
    targets = node_target_sequence
)

In [60]:
from torch_geometric_temporal.signal import temporal_signal_split

train, test = temporal_signal_split(data_iter, train_ratio=0.8)

In [61]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [62]:
for time, snapshot in enumerate(train):
    print(snapshot.x.shape)
    print(snapshot.edge_index.shape)
    print(snapshot.edge_attr.shape)
    print(snapshot.num_nodes)
    print(snapshot.num_edge_features)
    break

torch.Size([28, 3])
torch.Size([2, 189])
torch.Size([189, 1])
28
1


In [63]:
from tqdm import tqdm

model = RecurrentGCN(node_features=3, filters=2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(100)):
    for time, snapshot in enumerate(train):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()


100%|██████████| 100/100 [00:33<00:00,  2.97it/s]


In [64]:
model.eval()
cost = 0
for time, snapshot in enumerate(test):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("RMSE: {:.4f}".format(cost**0.5))

RMSE: 32.9165
